In [155]:
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [156]:
import re 
import os
import nltk
from nltk.tokenize import word_tokenize
from nltk import PorterStemmer
import math
import numpy as np
import pickle
from collections import Counter

In [157]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [158]:
os.getcwd()

'/content/gdrive/My Drive/CS-582_project'

In [159]:
path = "/content/gdrive/MyDrive/CS-582_project"
os.chdir(path)
os.getcwd()

'/content/gdrive/MyDrive/CS-582_project'

In [160]:
stopwords=nltk.corpus.stopwords.words('english')
ps = PorterStemmer()

In [161]:
# storing document lengths
web_page_length = {}

#calculating idf for tokens in web pages
web_page_idf = {}

# 

In [162]:
with open("Pickle_Folder/" + "6000_inverted_index.pickle", "rb") as f:
    inverted_index = pickle.load(f)

with open("Pickle_Folder/" + "6000_url_pages.pickle", "rb") as f:
    url_pages = pickle.load(f)


In [163]:
doc_count = len(url_pages)
doc_count

6001

Invert Document Frequency

In [164]:
def inverted_freq(data, inv_freq, count):
    for i in data.keys():
        inv_freq[i] = math.log(count/len(data[i]), 2) 

Calculating Document length.

In [165]:
def document_length(data, count, length, inv_freq):
    for i in data:
        for j in data[i].keys():
            if j not in length:
                length[j] = (data[i][j] * inv_freq[i])**2
            else:
                length[j] += (data[i][j] * inv_freq[i])**2
    return length

In [166]:
# inverted document frequency 
inverted_freq(inverted_index, web_page_idf, doc_count)

In [167]:
# calculating web page length
document_length(inverted_index, doc_count, web_page_length, web_page_idf)
# web_page_length = dict(sorted(web_page_length.items(), key=lambda x: x[0]))
web_page_length["1"]

704.2544651859436

Tokenize query data

In [168]:
def tokenize_query(data):
  data = data.lower()
  data = re.sub('[^a-z]+', ' ', data)                 
  words = data.split()
  stop_words_rem = [i for i in words if (len(i)>2 and i not in stopwords)]
  word_clean = [ps.stem(i) for i in stop_words_rem if len(ps.stem(i))>2]
  return word_clean

Query length

In [169]:
def query_length(data, inv_freq):

  length = 0

  for j in data.keys():
    length += (data[j] * inv_freq[j])**2
    
  return length


In [170]:
cosine_sim = {}
cosine_sim.keys()

dict_keys([])

Cosine Similarity

In [171]:
def cosine_similarity(query_tf):
  for j in query_tf:
      if j in inverted_index.keys():
          word_in_docs = list(inverted_index[j].keys())
          for k in word_in_docs:
            if k not in cosine_sim.keys():
              cosine_sim[k] = (query_tf[j]*web_page_idf.get(j,0)*inverted_index[j][k]*web_page_idf.get(j,0))/(math.sqrt(web_page_length[k]*query_len))
            else:
              cosine_sim[k] += (query_tf[j]*web_page_idf.get(j,0)*inverted_index[j][k]*web_page_idf.get(j,0))/(math.sqrt(web_page_length[k]*query_len))

  return cosine_sim
                        

Input query

In [172]:
print("\n                     ---UIC Web Search Engine---\n")
# query input 
query = str(input("Enter  search query: "))
print("\n")
# tokenize input query
tokenized_query = tokenize_query(query)


                     ---UIC Web Search Engine---

Enter  search query: Recreation Center




In [173]:
# calculating term frequency for given query
query_tf = Counter(tokenized_query)
# calculating query length
query_len = query_length(query_tf, web_page_idf)
query_len

35.36529188529818

In [174]:
# cosine similarity
cos_sim_val = cosine_similarity(query_tf)
sorted_relevant_pages = dict(sorted(cos_sim_val.items(), key=lambda x: x[1], reverse=True))

In [175]:
def display_relevant_pages(count, webpages):
    for i in range(count, count + 10):
      try:
        url_num = int(webpages[i])
        # print(url_no)
      
      # executed when there are no more relevant pages.
      except Exception as e:
        print("\nNo more relevant pages!!")
        break

      if url_pages.get(url_num, None):
        print(i + 1, url_pages.get(url_num))


In [176]:
inputs = {"yes", "y"}
# to implement do-while loop
first_pass = True
count = 0

while first_pass or answer.lower() in inputs:
    display_relevant_pages(count, list(sorted_relevant_pages.keys()))
    answer = str(input("\nDo you want more web page results? "))
    count += 10
    first_pass = False


1 https://recreation.uic.edu/memberships
2 https://recreation.uic.edu/facilities/facility_srf
3 https://recreation.uic.edu/memberships/campus-recreation-memberships
4 https://apps.sa.uic.edu/static/pcs/campusrec.html
5 https://recreation.uic.edu/news-stories/join-the-2020-national-recreation-movement
6 https://recreation.uic.edu
7 http://recreation.uic.edu
8 http://recreation.uic.edu/memberships/electronic-fund-transfer
9 https://sa.uic.edu/fall-reopening/campus-recreation
10 https://recreation.uic.edu/facilities/facility_sfc

Do you want more web page results? no
